In [24]:
using ParallelTemperingMonteCarlo
# using Pkg;Pkg.add("BenchmarkTools")
using BenchmarkTools
using Random,StaticArrays

# Equilibration

The purpose of this notebook is to write functions aimed at equilibrating the ptmc run. It must iterate over every trajectory for, by default 20\% of the total cycles run with _no_ sampling, other than testing the $\geq$ condition to update the min/max energies

In [25]:
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 1000000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

first, the $\geq$ condition: test whether it's faster to unpack mc_states or not. 

In [26]:
function testmin(new_en,en_min)
    if new_en < en_min
        en_min = new_en
    end
    return en_min
end

function test_min(new_en,en_min)
    return ifelse(new_en<en_min,new_en,en_min)
end
function test_max(new_en,en_max)
    return ifelse(new_en>en_max,new_en,en_max)
end

test_max (generic function with 1 method)

In [27]:
@benchmark testmin($rand(),$rand())

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.461 ns … 4.832 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.476 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.508 ns ± 0.116 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▅█             ▁▁                                        
  ▅███▅▂▁▁▁▁▁▁▁▁▁▁▇██▇▃▁▁▁▁▁▁▁▁▁▁▁▂▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  2.46 ns        Histogram: frequency by time       2.65 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [28]:
@benchmark test_min($rand(),$rand())

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.460 ns … 4.222 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.525 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.542 ns ± 0.096 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▁         ▄█▁         ▃▆                                 
  ▅██▅▂▃▂▂▂▂▂▄███▃▄▅▄▃▂▃▂▅███▃▃▃▃▂▂▂▂▂▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂ ▃
  2.46 ns        Histogram: frequency by time       2.72 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Use the ifelse version, it's slightly faster more often. NB the function guarantees type stability, telling the compiler what is to be returned, this is better.

In [29]:
function check_e_bounds(energy,ebounds::Vector)
    ebounds[1] = test_min(energy,ebounds[1])
    ebounds[2] = test_max(energy,ebounds[2])
    return ebounds
end
function checkbounds2(energy,emin,emax)
    emin = test_min(energy,emin)
    emax = test_max(energy,emax)
    return emin,emax
end
function checkbounds3(energy,ebounds)
    if energy<ebounds[1]
        ebounds[1]=energy
    elseif energy>ebounds[2]
        ebounds[2] = energy
    else
    end
    return ebounds
end
function checkbounds4(energy,ebounds)
    ebounds[1],ebounds[2] = test_min(energy,ebounds[1]),test_max(energy,ebounds[2])
    return ebounds 
end

checkbounds4 (generic function with 1 method)

In [30]:
ebounds = [-1.,1.]
energy = 0.6
ebounds = check_e_bounds(energy,ebounds)
@benchmark check_e_bounds($rand(-1.5:1.5),$ebounds)

BenchmarkTools.Trial: 10000 samples with 956 evaluations.
 Range (min … max):   94.928 ns … 147.021 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     100.297 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   100.320 ns ±   2.404 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                    ▁▁        ▄▆██▄▁                             
  ▁▁▁▂▂▃▃▄▄▃▂▂▁▂▃▄▆███▇▆▄▃▃▃▅███████▆▅▄▄▄▄▅▆▆▆▅▅▄▃▂▂▂▂▂▂▂▃▂▂▂▂▁ ▃
  94.9 ns          Histogram: frequency by time          106 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [31]:
ebounds = [-1.,1.]
@benchmark checkbounds2($rand(-1.5:1.5),$ebounds[1],$ebounds[2])

BenchmarkTools.Trial: 10000 samples with 956 evaluations.
 Range (min … max):  90.208 ns … 124.110 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     94.566 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   94.622 ns ±   2.088 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▁▃▃▂▁      ▁▃▅▇▇▅▃▂    ▁▃▅▇▆█▆▅▂▁                     
  ▁▁▁▂▂▄▅▇██████▆▅▅▅▇█████████▇▆███████████▇▆▆▆▅▄▅▅▄▅▄▃▃▃▂▂▂▂▂ ▅
  90.2 ns         Histogram: frequency by time         99.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [32]:
ebounds = [-1.,1.]
energy = 0.6
@benchmark checkbounds3($rand(-1.5:1.5),$ebounds)

BenchmarkTools.Trial: 10000 samples with 956 evaluations.
 Range (min … max):  94.744 ns … 124.756 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     99.778 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   99.338 ns ±   1.839 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                       ▁▂▁▁         ▃▆▇█▇▆▃                     
  ▁▁▁▁▂▃▄▄▅▅▄▄▃▂▂▂▂▃▄▆▇████▇▅▄▄▃▃▄▆█████████▆▅▄▃▃▃▃▃▃▃▃▂▃▂▂▂▁▁ ▃
  94.7 ns         Histogram: frequency by time          103 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [33]:
ebounds = [-1.,1.]
energy = 0.6
ebounds = checkbounds4(energy,ebounds)
@benchmark checkbounds4($rand(-1.5:1.5),$ebounds)

BenchmarkTools.Trial: 10000 samples with 958 evaluations.
 Range (min … max):  94.914 ns … 129.341 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     98.940 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   99.071 ns ±   1.954 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▂▃▁         ▂▄█▇▄▅▁       ▂▆▇▇█▅▃▁                    
  ▁▁▁▂▃▄▆█████▇▅▄▃▃▄▅▇███████▇▅▅▅▅▆█████████▇▆▅▄▄▃▃▃▃▃▂▂▂▂▁▂▂▁ ▄
  94.9 ns         Histogram: frequency by time          104 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.